# Model comparison

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression
from fair_models import calc_reweight
from fair_models import FairScalarization, EqualScalarization

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


## Parte 1: Data treatment

In [3]:
dataset = pd.read_csv("Datasets - Originais/adult.csv")

dataset = dataset.fillna(np.nan)

dataset['income']=dataset['income'].map({'<=50K': -1, '>50K': 1, '<=50K.': -1, '>50K.': 1})

In [4]:
# Fill Missing Category Entries
dataset["workclass"] = dataset["workclass"].fillna("X")
dataset["occupation"] = dataset["occupation"].fillna("X")
dataset["native.country"] = dataset["native.country"].fillna("United-States")

In [5]:
# Convert Sex value to 0 and 1
dataset["sex"] = dataset["sex"].map({"Male": 0, "Female":1})
dataset["race"] = dataset["race"].map({"White": 0, "Black":1})

# Create Married Column - Binary Yes(1) or No(0)
dataset["marital.status"] = dataset["marital.status"].replace(['Never-married','Divorced','Separated','Widowed'], 'Single')
dataset["marital.status"] = dataset["marital.status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'], 'Married')
dataset["marital.status"] = dataset["marital.status"].map({"Married":1, "Single":0})
dataset["marital.status"] = dataset["marital.status"].astype(int)

# Drop the data you don't want to use
dataset.drop(labels=["workclass","education","occupation","relationship","native.country"], axis = 1, inplace = True)

dataset = dataset.dropna()

In [6]:
dataset.head()

,age,fnlwgt,education.num,marital.status,race,sex,capital.gain,capital.loss,hours.per.week,income
0,90,77053,9,0,0.0,1,0,4356,40,-1
1,82,132870,9,0,0.0,1,0,4356,18,-1
2,66,186061,10,0,1.0,1,0,4356,40,-1
3,54,140359,4,0,0.0,1,0,3900,40,-1
4,41,264663,10,0,0.0,1,0,3900,40,-1


In [7]:
X = dataset.drop(['income'], axis=1)

In [8]:
y = dataset['income']

In [9]:
dataset.to_csv("Datasets - Limpos/adult.csv")

In [8]:
X.shape

(30940, 9)

In [9]:
# 395, 808
random_seed = 2000#np.random.randint(0, 1000)
random_seed

2000

In [10]:
random_seed2 = 2000#np.random.randint(0, 1000)
random_seed2

2000

In [11]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=8000, random_state = random_seed)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=4000, random_state = random_seed2)

In [12]:
fair_feat = 'race'

In [13]:
sample_weight = calc_reweight(X_train, y_train, fair_feat)

## Parte 3: Comparando os modelos

In [14]:
optimize_metrics = {'accuracy': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []}}

In [12]:
metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']

In [16]:
methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo', 'RegEqMoo']
index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)
results_val = pd.DataFrame(index=index, columns=metrics)

In [17]:
fair_feature = "race"

In [18]:
# RegLog

for metric in metrics:
    reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric).tune()
    # RegMoo
    optimize_metrics[metric]['LogReg'] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]
    results_test.loc[('LogReg',metric),:] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]

In [19]:
# RegEqual

for metric in metrics:
    reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric, base_model='equal').tune()

    optimize_metrics[metric]['RegEqual'] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]
    results_test.loc[('RegEqual',metric),:] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


In [20]:
# RegDemo

for metric in metrics:
    reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val,fair_feature, metric=metric, base_model='demographic').tune()

    optimize_metrics[metric]['RegDemo'] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]
    results_test.loc[('RegDemo',metric),:] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

In [21]:
# RegRewe

for metric in metrics:
    reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature,metric=metric, sample_weight=sample_weight).tune()

    optimize_metrics[metric]['RegRewe'] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]
    results_test.loc[('RegRewe',metric),:] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]

In [22]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_moo = moo_learn.tune(metric)
    
    results_test.loc[('RegMoo',metric),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

Using license file /opt/gurobi/gurobi.lic
Academic license - for non-commercial use only


/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

In [23]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_eqmoo = moo_learn.tune(metric)
    
    results_test.loc[('RegEqMoo',metric),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

In [24]:
results_test

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy            0.80125          0.925735   0.56058    0.697731
         equal_opportunity  0.798875          0.969916  0.541356    0.704289
         p_percent          0.800875          0.896911  0.555308    0.698798
         c_variation        0.801125          0.925735  0.559733    0.697947
RegEqual accuracy             0.8445          0.856511  0.559733    0.583645
         equal_opportunity     0.821          0.972102  0.559733    0.658731
         p_percent           0.82125          0.988129  0.559733      0.6582
         c_variation          0.8445          0.856511  0.559733    0.583645
RegDemo  accuracy             0.8445          0.856511  0.417323    0.583645
         equal_opportunity    0.8355          0.974004  0.539689    0.616088
         p_percent            0.7905          0.845879    0.9871     0.71668
         c_variation          0.8445          0.856511  0.417323    0.583645
RegRewe  accuracy           0.801375          0.935668  0.555308    0.697811
         equal_opportunity  0.807625          0.822896  0.606422    0.689217
         p_percent          0.800625           0.89307  0.564853    0.699111
         c_variation        0.801375          0.935668  0.555308    0.697811
RegMoo   accuracy            0.80175          0.954379  0.598418    0.698779
         equal_opportunity     0.804          0.781198  0.624221    0.698159
         p_percent            0.8035          0.930155  0.565863     0.69604
         c_variation        0.801125          0.935668  0.553607    0.698245
RegEqMoo accuracy             0.8015          0.935668  0.556163    0.697593
         equal_opportunity    0.2345                 1         1    0.339384
         p_percent            0.2345                 1         1    0.339384
         c_variation          0.2345                 1         1    0.339384

In [25]:
def dominate(a, b):
    sense = np.array([1, 1, 1, -1])
    if all((sense*a)>=(sense*b)) and any((sense*a)>(sense*b)):
        return True
    else:
        return False

In [26]:
new_results = results_test.copy()//0.01/100
dominate_metr = [any([dominate(other, row) for other in new_results.values]) for row in new_results.values]
new_results['dominated'] = dominate_metr
new_results

accuracy equal_opportunity p_percent c_variation  \
LogReg   accuracy              0.61               0.9      0.96        0.67   
         equal_opportunity     0.56              0.99      0.98        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegEqual accuracy              0.58              0.95      0.99        0.57   
         equal_opportunity     0.54              0.99      0.99        0.48   
         p_percent             0.61              0.95      0.99         0.7   
         c_variation           0.53              0.99      0.99        0.48   
RegDemo  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.61              0.94      0.92        0.71   
         c_variation           0.53              0.99      0.99        0.48   
RegRewe  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegMoo   accuracy              0.59              0.96      0.93        0.56   
         equal_opportunity     0.59              0.96      0.93        0.56   
         p_percent             0.56              0.98      0.95        0.77   
         c_variation           0.58              0.98      0.98        0.51   

                            dominated  
LogReg   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation             True  
RegEqual accuracy               False  
         equal_opportunity      False  
         p_percent              False  
         c_variation             True  
RegDemo  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegRewe  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegMoo   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation            False

In [27]:
print(new_results.to_latex())

\begin{tabular}{lllllll}
\toprule
       &             & accuracy & equal\_opportunity & p\_percent & c\_variation &  dominated \\
\midrule
LogReg & accuracy &     0.61 &               0.9 &      0.96 &        0.67 &      False \\
       & equal\_opportunity &     0.56 &              0.99 &      0.98 &        0.48 &      False \\
       & p\_percent &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegEqual & accuracy &     0.58 &              0.95 &      0.99 &        0.57 &      False \\
       & equal\_opportunity &     0.54 &              0.99 &      0.99 &        0.48 &      False \\
       & p\_percent &     0.61 &              0.95 &      0.99 &         0.7 &      False \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegDemo & accuracy &     0.58 &              0.95 &      0.92 &        0.57 &       True \\
      

## Models with ensemble

In [25]:
ensemble_metrics = {'accuracy': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []}}

In [10]:
ensembles = ['voting soft', 'voting hard', 'knorau', 'knorae']

In [13]:
methods = ['RegMoo', 'RegEqMoo']
index = [sum([[method]*len(ensembles) for method in methods],[]),
        sum([ensembles for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)

In [14]:
results_test

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft      NaN               NaN       NaN         NaN
         voting hard      NaN               NaN       NaN         NaN
         knorau           NaN               NaN       NaN         NaN
         knorae           NaN               NaN       NaN         NaN
RegEqMoo voting soft      NaN               NaN       NaN         NaN
         voting hard      NaN               NaN       NaN         NaN
         knorau           NaN               NaN       NaN         NaN
         knorae           NaN               NaN       NaN         NaN

In [15]:
for ensemble in ensembles:
    for method in methods:
        for i in range(4):
            results_test.loc[(method,ensemble),metrics[i]] = 0

In [16]:
results_test

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft        0                 0         0           0
         voting hard        0                 0         0           0
         knorau             0                 0         0           0
         knorae             0                 0         0           0
RegEqMoo voting soft        0                 0         0           0
         voting hard        0                 0         0           0
         knorau             0                 0         0           0
         knorae             0                 0         0           0

In [28]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)

for ensemble in ensembles:
    reg_moo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegMoo',ensemble),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

In [29]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)

for ensemble in ensembles:
    reg_eqmoo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegEqMoo',ensemble),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

In [30]:
results_test

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.802125          0.963196  0.593144    0.698296
         voting hard  0.802125           0.98203  0.585495    0.697695
         knorau         0.8035          0.924754  0.568593    0.694909
         knorae         0.8015          0.987173  0.550426    0.701251
RegEqMoo voting soft   0.79975          0.919876  0.559854    0.700115
         voting hard  0.800875          0.898844  0.542258    0.698856
         knorau       0.800875          0.898844  0.542258    0.698856
         knorae       0.696375          0.981145  0.777431    0.754938